In [1]:
# if you want to use on COLAB upload PDF files to your drive and set patth correctly my folder is "pdfs"
# else commend this cell
from google.colab import drive
drive.mount('/content/drive')
!cp drive/MyDrive/pdfs/test.pdf test.pdf

Mounted at /content/drive


In [2]:
!pip install pytesseract pdf2image langdetect
!pip install opencv-python==4.5.1.48
!sudo apt install tesseract-ocr
!apt-get install poppler-utils
!apt-get install tesseract-ocr-fra

     |████████████████████████████████| 983kB 12.9MB/s 
  Created wheel for pytesseract: filename=pytesseract-0.3.7-py2.py3-none-any.whl size=13945 sha256=bdb8a6b3955e85e3935e7fcc48a64ac978820aec0fd5c165e06ba70e1616601b
  Stored in directory: /root/.cache/pip/wheels/81/20/7e/1dd0daad1575d5260916bb1e9781246430647adaef4b3ca3b3
  Created wheel for langdetect: filename=langdetect-1.0.8-cp37-none-any.whl size=993193 sha256=477ebef73e33aa51d9ef1d357e11f0eadbf75ce4e14c1cec9f1853c7cc2d9593
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
Successfully built pytesseract langdetect
     |████████████████████████████████| 50.4MB 87kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
Reading package lists... Done
Building de

In [1]:
from pdf2image import convert_from_path 
from pytesseract import Output 
from langdetect import detect
from PIL import Image 
import pandas as pd
import numpy as np
import pytesseract 
import sys 
import cv2
import os 
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

In [4]:
# !ls

In [2]:
def pdf_to_images(PDF_file) :
    pages = convert_from_path(PDF_file, 300)  # DPI
    image_counter = 1
    for page in pages:
      filename = PDF_file+"_"+str(image_counter)+".jpg"
      page.save(filename, 'JPEG') 
      image_counter += 1
    return pages

def is_french(stringx) :
    try :
        if detect(stringx.lower()) == 'fr' :
            return True
    except :
        return False

In [6]:
################################# We can adapt this code to pipeline############################################
        # text_list = text_to_list(text)
        # for line in text_list :
        #   if is_french(line) or line.startswith('Artic') or line.startswith('§')  :
        #     # if is_french(line) or line.startswith('Artic')  :
        #     fr_list.append(line.replace('\n',' '))

In [3]:
##### Functions for the Pipeline start here #####
# 1 - call English NLP and store results to a variable (because it finds § symbol perfectly)
def en_nlp_for_symbol(filename):  
  d = pytesseract.image_to_data(filename, output_type=Output.DICT)
  return d.copy()

In [4]:
# 2 - call French NLP and store results to another variable
def fr_nlp_for_symbol(filename):  
  d = pytesseract.image_to_data(filename, output_type=Output.DICT, lang="fra")
  return d.copy()

In [5]:
### if you like you can SAVE and SEE boxed PDF image for fr_nlp and en_nlp USE this code ###
### n_boxes = len(d['level'])
### for i in range(n_boxes):
###   if(d['text'][i] != ""):
###     (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
###     cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
### cv2.imwrite('orhan2.png', img) ###

In [6]:
# 3 - from English NLP results, system collect all § symbols coordinates and index info in a list
def info_from_en_nlp_symbol(en_nlp, symbol="§"):
  pos_list=[]         # list has records in a tuple (index of English NLP, [coordinates,top,left,...])    
  for t in range(len(en_nlp['text'])):
    if symbol in en_nlp['text'][t]:
      pos_list.append([t,[en_nlp['left'][t], en_nlp['top'][t], en_nlp['width'][t], en_nlp['height'][t]]])
  return pos_list

In [7]:
# 4 - match all § symbols coordinates with French NLP results and make a list
def match_lists_en_and_fr_nlps(fr_nlp,pos_list, tolerans=1):   # give one pixel tolerans to match Fr and En NLPs
  match_list=[]  # x[0] = left ,  x[1] = top , x[2] = width , x[3] = height ,
  for k in pos_list:#for k,(a,x) in enumerate(pos_list):
    for l0 in range(len(fr_nlp["left"])):  
      if ( fr_nlp["left"][l0] + tolerans > k[1][0] ) and ( fr_nlp["left"][l0] - tolerans < k[1][0] ):
        if ( fr_nlp["top"][l0] + tolerans > k[1][1] ) and ( fr_nlp["top"][l0] - tolerans < k[1][1] ):
          if ( fr_nlp["width"][l0] + tolerans > k[1][2] ) and ( fr_nlp["width"][l0] - tolerans < k[1][2] ):
            if ( fr_nlp["height"][l0] + tolerans > k[1][3] ) and ( fr_nlp["height"][l0] - tolerans < k[1][3] ):            
              match_list.append([l0,[fr_nlp['left'][l0], fr_nlp['top'][l0], fr_nlp['width'][l0], fr_nlp['height'][l0]]])#print(fr_nlp["text"][l0]," , index = ",l0,fr_nlp["left"][l0],fr_nlp["top"][l0],fr_nlp["width"][l0],fr_nlp["height"][l0])
  return match_list

In [8]:
# 5 - Delete all duplicates from matched boxes list
def delete_dublicates_from_list(match_list):
  delete_list=[]
  for k in match_list:
    cont_list=[]
    for k1 in match_list:
      if k[1]==k1[1]:
        cont_list.append(k1[0])
    for element in cont_list:
      if max(cont_list)!= element: 
        delete_list.append(element)
  for d in list(dict.fromkeys(delete_list)):
    for m in match_list:
      if m[0] == d:
        match_list.pop(match_list.index(m))  # print(len(match_list),match_list)
  return match_list

In [9]:
# 7 - call is French function to filter to get just French results
def df_fr_sentences(df_fr):
  df_fr_filtered = pd.DataFrame()
  df_fr_filtered["id"]=list(df_fr.index.values)
  df_fr_filtered["fullsentence"] =df_fr.groupby(['block_num','par_num'])['text'].transform(' '.join)
  df_fr_filtered["fullsentence"] = df_fr_filtered["fullsentence"].str.lstrip(' ')
  for i in range(20):
    df_fr_filtered["fullsentence"] = df_fr_filtered["fullsentence"].replace(["      ","     ","    ","   ","  "],"")
  # left	top	width	height	conf	text	fullsentence
  # for x in  df_fr_filtered:
  #   if(x['left'])
  # df_fr_filtered["fullsentence"].startswith('§') ve left	top	width	height esit ise sadece max al diger sil

  df_fr_filtered = df_fr_filtered.drop(df_fr_filtered[df_fr_filtered["fullsentence"] == ""].index , axis=0)
  df_fr_filtered = df_fr_filtered.drop(df_fr_filtered[df_fr_filtered["fullsentence"] == " "].index , axis=0)
  # check before and later paragraphs if they in NL skip that symbol
  # check before and later paragraphs if they in NL skip that art
  nl_sen = pd.Series([True if (is_french(x) or x.startswith('Art') or x.startswith('§')) else False for x in df_fr_filtered["fullsentence"]])

  return df_fr_filtered.loc[nl_sen.fillna(False).values]

In [10]:
### PIPELINE OF MERGING TWO NLP RESULTS TO BE ABLE TO GET FRENCH CHARACTERSET AND § SYMBOL

def fill_fr_nlp_from_en_nlp(image_name):
  # 1 - call English NLP and store results to a variable (because it finds § symbol perfectly)
  en_nlp = en_nlp_for_symbol(image_name) # 300 dpi is recommended for OCR image files.
  df_en = pd.DataFrame(en_nlp)  

  # 2 - call French NLP and store results to another variable
  fr_nlp = fr_nlp_for_symbol(image_name)
  df_fr = pd.DataFrame(fr_nlp)  

  # 3 - from English NLP results, system collect all § symbols coordinates and index info in a list
  pos_en = info_from_en_nlp_symbol(en_nlp)

  # 4 - match all § symbols coordinates with French NLP results and make a list
  pos_fr = match_lists_en_and_fr_nlps(fr_nlp, pos_en)

  # 5 - Delete all duplicates from match list
  pos_fr = delete_dublicates_from_list(pos_fr)

  # 6 - Change French results with desired characters
  for m in pos_fr:
    df_fr["text"][m[0]] = df_en["text"][pos_en[pos_fr.index(m)][0]]
  
  # 7 - call is French function to filter to get just French results
  
  df_fr_sent = df_fr_sentences(df_fr)
  
  df_fr["id"] = df_fr.index.values
  df_fr_merged = pd.merge(df_fr, df_fr_sent, how='inner', on=['id'])
  
  return df_fr_merged, df_fr #, df_en , match_list, pos_list

In [11]:
pages = pdf_to_images("test.pdf") # this cell and next cell can merge to have a function

PDFPageCountError: Unable to get page count.
I/O Error: Couldn't open file 'test.pdf': No such file or directory.


In [16]:
df_fr_merged_full = pd.DataFrame(columns = ['id','level', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num', 'left', 'top', 'width', 'height', 'conf', 'text'])
df_fr_full = pd.DataFrame(columns = ['id','level', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num', 'left', 'top', 'width', 'height', 'conf', 'text'])
print(f"this document has {len(pages) } pages the process has started ..."
# for page in page    s:
df_fr_merged, df_fr = fill_fr_nlp_from_en_nlp(pages[0]    )  
del df_fr_merged["page_num"    ]   
del df_fr["page_num    "]   
df_fr_merged["page_num"] = pages.index(pages[0]    ) + 1 
df_fr["page_num"] = pages.index(pages[0    ]) + 1 
df_fr_merged_full = df_fr_full.append(df_fr    _merged)
df_fr_full = df_fr_full.appe    nd(df_fr)
print(f"{pages.index(pages[0]) + 1}. page is     finished")
df_fr_merged_full = df_fr_merged_full.reset_index    (drop=True)
df_fr_full = df_fr_full.reset_index(drop=True)

this document has 7 pages the process has started ...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1. page is finished


In [17]:
# df_fr_full[[True if (('-1'==x["conf"]) and (" " not in x["text"])) else False for x in df_fr_full]]
[True if (" " not in x) else False for x in df_fr_full[[True if (('-1'==x)) else False for x in df_fr_full["conf"]]]["text"]]
# df_fr_full[["LA" in x for x in df_fr_full["text"]]]


[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,

In [18]:
# [x.values for x in df_fr_full]

In [19]:
df_fr_merged_full[["§1" == x for x in df_fr_merged_full["fullsentence"]]]
# df_fr_filtered["fullsentence"].startswith('§') ve left	top	width	height esit ise sadece max al diger sil
  
  
# df_fr_merged_full[[x==1716 for x in df_fr_merged_full["top"]]]
# df_fr_merged_full[139:149]

,id,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,fullsentence
69,280,3,1,15,1,0,0,295,1920,38,42,-1,,§1
70,281,4,1,15,1,1,0,295,1920,38,42,-1,,§1
71,282,5,1,15,1,1,1,295,1920,38,42,75,§1,§1
138,514,3,1,26,1,0,0,1249,1921,40,41,-1,,§1
139,515,4,1,26,1,1,0,1249,1921,40,41,-1,,§1
140,516,5,1,26,1,1,1,1249,1921,40,41,36,§1,§1


In [20]:

df_para = pd.DataFrame()
df_para["paragraph"] = df_fr_full.groupby(['page_num','block_num','par_num'])['text'].transform(' '.join).str.lstrip().str.rstrip().unique()
for i in range(20):
  df_para["paragraph"] = df_para["paragraph"].str.lstrip()
para_list = df_para["paragraph"].unique()
para_list[3:5]


array(['HET AANVULLENDE PARITAIR COMITE  VOOR DE ARBEIDERS TOT INSTELLING  EN VASTSTELLING VAN DE PROCEDURE',
       'VAN TENUITVOERLEGGING EN VAN DE  VOORWAARDEN VOOR DE TOEKENNING'],
      dtype=object)

In [21]:
df_fr_full.groupby(['page_num'])['text'].transform(' '.join).str.lstrip().str.rstrip().unique()

array(["Neerlegging-Dépôt: 27/11/2019  Regist.-Enregistr.: 04/02/2020  N°: 156733/C0/100         COLLECTIEVE  ARBEIDSOVEREENKOMST VAN  13 NOVEMBER 2019 AFGESLOTEN IN   HET AANVULLENDE PARITAIR COMITE  VOOR DE ARBEIDERS TOT INSTELLING  EN VASTSTELLING VAN DE PROCEDURE   VAN TENUITVOERLEGGING EN VAN DE  VOORWAARDEN VOOR DE TOEKENNING   BETREFFENDE HET STELSEL VAN  WERKLOOSHEIÏD MET  BEDRIJFSTOESLAG IN UITVOERING VAN  CAO NR 138 en NR 139 GESLOTEN IN DE  NATIONALE ARBEIDSRAAD                                         CONVENTION COLLECTIVE DE TRAVAIL  DU 13 NOVEMBRE 2019 CONCLUE AU  SEIN DE LA COMMISSION PARITAIRE   AUXILIAIRE POUR OUVRIERS  INSTAURANT ET DETERMINANTLA  PROCEDURE DE MISE EN ŒUVRE ET  LES CONDITIONS D'OCTROI D'UN  REGIME DE CHOMAGE AVEC  COMPLEMENT D'ENTREPRISE EN  EXECUTION DE LA CCT n° 138 et n° 139  CONCLUE AU SEIN DU CONSEIL  NATIONAL DU TRAVAIL                   HOOFDSTUK I — TOEPASSINGSGEBIED  Artikel 1                                       Deze collectieve arbeidsovere

In [22]:
i=0
for a in df_fr_full.groupby(['block_num','par_num'])['text'].transform(' '.join)[["§" in x for x in df_fr_full.groupby(['block_num','par_num'])['text'].transform(' '.join)]]:
  if i<7:
    print(len(a),a,a.index("§"),len(a.lstrip(' ')))
  i+=1

4   §1 2 2
4   §1 2 2
4   §1 2 2
4   §2 2 2
4   §2 2 2
4   §2 2 2
157   Zij wordt gesloten ingevolge artikel 3, § 1 van  het koninklijk besluit van 3 mei 2007 tot  regeling van het stelsel van werkloosheid met  bedrijfstoeslag. 42 155


In [23]:
##### now it is turn for this 5 last tasks #####
# 1 - call structuring functions to add new columns to the dataset      # Almost finished
# 2 - call storeDB function to save everything to the DB                # a few lines
# 3 - call function to update DB with CLA relations and changes         # a few lines
# 4 - call QnA system to save desired questions and answers to the DB   # a few lines
# 5 - call API to present the project to the user                       # will take time
# 6 - Presentation Government Website is our template                   # will take time because you did greate job guys show it
df_fr_merged_full.groupby(['page_num'])['text'].transform(' '.join).str.lstrip().str.rstrip().unique()

array(["CONVENTION COLLECTIVE DE TRAVAIL  DU 13 NOVEMBRE 2019 CONCLUE AU  SEIN DE LA COMMISSION PARITAIRE   AUXILIAIRE POUR OUVRIERS  INSTAURANT ET DETERMINANTLA  PROCEDURE DE MISE EN ŒUVRE ET  LES CONDITIONS D'OCTROI D'UN  REGIME DE CHOMAGE AVEC  COMPLEMENT D'ENTREPRISE EN  EXECUTION DE LA CCT n° 138 et n° 139  CONCLUE AU SEIN DU CONSEIL  NATIONAL DU TRAVAIL   §1   §2   §3   CHAPITRE | - CHAMP D'APPLICATION  Article 1   La présente convention collective de travail  s'applique aux employeurs et aux ouvriers  ressortissants du champ d'application de la  commission paritaire auxiliaire pour ouvriers.  Par «ouvriers », on entend les ouvriers  masculins et féminins.   - PORTEE LA  CONVENTION  Article 2   §1   La présente convention a aussi bien pour objet  d'instaurer un régime d'indemnité  complémentaire applicable à certains   travailleurs âgés licenciés que d'en déterminer  la procédure de mise en œuvre et les  conditions d'octroi.   §2   Elle est conclue en vertu de l'article 3, § 1er 

In [26]:
df_fr_merged_full["fullsentence"].unique()

array(['CONVENTION COLLECTIVE DE TRAVAIL  DU 13 NOVEMBRE 2019 CONCLUE AU  SEIN DE LA COMMISSION PARITAIRE',
       "AUXILIAIRE POUR OUVRIERS  INSTAURANT ET DETERMINANTLA  PROCEDURE DE MISE EN ŒUVRE ET  LES CONDITIONS D'OCTROI D'UN  REGIME DE CHOMAGE AVEC  COMPLEMENT D'ENTREPRISE EN  EXECUTION DE LA CCT n° 138 et n° 139  CONCLUE AU SEIN DU CONSEIL  NATIONAL DU TRAVAIL",
       '§1', '§2', '§3', "CHAPITRE | - CHAMP D'APPLICATION  Article 1",
       "La présente convention collective de travail  s'applique aux employeurs et aux ouvriers  ressortissants du champ d'application de la  commission paritaire auxiliaire pour ouvriers.  Par «ouvriers », on entend les ouvriers  masculins et féminins.",
       'CHAPITRE Il - PORTEE DE LA  CONVENTION  Article 2',
       "La présente convention a aussi bien pour objet  d'instaurer un régime d'indemnité  complémentaire applicable à certains",
       "travailleurs âgés licenciés que d'en déterminer  la procédure de mise en œuvre et les  conditions d'oc

In [27]:
df_fr_full

,id,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
0,0,1,1,0,0,0,0,0,0,2488,3513,-1,
1,1,2,1,1,0,0,0,1228,333,12,5,-1,
2,2,3,1,1,1,0,0,1228,333,12,5,-1,
3,3,4,1,1,1,1,0,1228,333,12,5,-1,
4,4,5,1,1,1,1,1,1228,333,12,5,95,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
679,679,5,1,34,1,1,1,2476,2909,12,48,95,
680,680,2,1,35,0,0,0,2470,2364,8,1149,-1,
681,681,3,1,35,1,0,0,2470,2364,8,1149,-1,
682,682,4,1,35,1,1,0,2470,2364,8,1149,-1,
